In [2]:
import pandas as pd

In [3]:
airports_df = pd.read_csv('../downloads/airports.csv')
airports_df.head()

,index,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Type,Source
0,0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports


In [4]:
airports_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7698 entries, 0 to 7697
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  7698 non-null   int64  
 1   Airport ID             7698 non-null   int64  
 2   Name                   7698 non-null   object 
 3   City                   7649 non-null   object 
 4   Country                7698 non-null   object 
 5   IATA                   7698 non-null   object 
 6   ICAO                   7698 non-null   object 
 7   Latitude               7698 non-null   float64
 8   Longitude              7698 non-null   float64
 9   Altitude               7698 non-null   int64  
 10  Timezone               7698 non-null   object 
 11  DST                    7698 non-null   object 
 12  Tz database time zone  7698 non-null   object 
 13  Type                   7698 non-null   object 
 14  Source                 7698 non-null   object 
dtypes: f

In [5]:
routes = pd.read_csv('../downloads/routes.csv')
routes.head()

,index,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment
0,0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2


In [6]:
routes[routes["Source airport"] == "MUC"]

,index,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment
433,433,3L,2916,MUC,346,EBA,1558,NaN,0,DH3
1514,1514,4U,2548,MUC,346,DTM,373,NaN,0,319
4407,4407,A3,96,MUC,346,ATH,3941,NaN,0,320
4408,4408,A3,96,MUC,346,HER,1452,NaN,0,320
4409,4409,A3,96,MUC,346,KLX,1456,NaN,0,319
...,...,...,...,...,...,...,...,...,...,...
65321,65321,X3,2681,MUC,346,TFS,1056,NaN,0,73H
65322,65322,X3,2681,MUC,346,XRY,1226,NaN,0,73H
65532,65532,XQ,4573,MUC,346,ADB,1706,NaN,0,738
65533,65533,XQ,4573,MUC,346,AYT,1688,NaN,0,738


In [7]:
routes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67663 entries, 0 to 67662
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   index                   67663 non-null  int64 
 1   Airline                 67663 non-null  object
 2   Airline ID              67663 non-null  object
 3   Source airport          67663 non-null  object
 4   Source airport ID       67663 non-null  object
 5   Destination airport     67663 non-null  object
 6   Destination airport ID  67663 non-null  object
 7   Codeshare               14597 non-null  object
 8   Stops                   67663 non-null  int64 
 9   Equipment               67645 non-null  object
dtypes: int64(2), object(8)
memory usage: 5.2+ MB


In [8]:
# Access the value of country of the airport "MUC" from the airports_df DataFrame
airports_df[airports_df["IATA"] == "MUC"]["Country"]

# Access the value of country of the airport "MUC" from the airports_df DataFrame
airports_df[airports_df["IATA"] == "MUC"]["Country"].values[0]

    

'Germany'

In [9]:
if source_country == "Germany":
    print("Its Germany")

NameError: name 'source_country' is not defined